In [1]:
# import relevant modules
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import imblearn
from sklearn.preprocessing import StandardScaler
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Settings
pd.set_option('display.max_columns', None)
#np.set_printoptions(threshold=np.nan)
np.set_printoptions(precision=3)
sns.set(style="darkgrid")
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1y98lxMqnA9Zc8zdHhaSeL3_zCUGy0bqR' # The Shareable link Preprocessing.csv

In [0]:
link2 = 'https://drive.google.com/open?id=1Qyl54uAdmET0ICQk4_mOwL1kk_Z5lO56' # The Shareable link Train_data.csv

In [5]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1y98lxMqnA9Zc8zdHhaSeL3_zCUGy0bqR


In [6]:
fluff2, id2 = link2.split('=')
print (id2) # Verify that you have everything after '='

1Qyl54uAdmET0ICQk4_mOwL1kk_Z5lO56


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Preprocessing.csv')  
df = pd.read_csv('Preprocessing.csv',delimiter=',')

In [0]:
downloaded2 = drive.CreateFile({'id':id2}) 
downloaded2.GetContentFile('Train_data.csv')  
df2 = pd.read_csv('Train_data.csv',delimiter=',')

In [0]:
df.shape

(25192, 37)

In [0]:
df2.drop(['num_outbound_cmds'], axis=1, inplace=True)
df2.drop(['protocol_type'], axis=1, inplace=True)
df2.drop(['service'], axis=1, inplace=True)
df2.drop(['flag'], axis=1, inplace=True)

In [0]:
df2.shape
class_label = df2['class']

In [0]:
class_label

0         normal
1         normal
2        anomaly
3         normal
4         normal
          ...   
25187    anomaly
25188    anomaly
25189    anomaly
25190    anomaly
25191    anomaly
Name: class, Length: 25192, dtype: object

In [0]:
df_with_label=pd.concat([df,class_label],axis=1)
df_with_label.columns = df2.columns

In [0]:
df_without_label = df_with_label.copy()
df_without_label.drop(['class'], axis=1, inplace=True)

In [0]:
df_without_label.shape

(25192, 37)

In [13]:
df_with_label['class'].value_counts()

normal     13449
anomaly    11743
Name: class, dtype: int64

In [0]:
train_x = df_without_label
train_y = class_label

In [0]:
from sklearn.model_selection import train_test_split
#split data yang akan digunakan untuk train dan test data 
#dengan perbandingan 70:30 ( train dan test )
#test disini digunakan untuk menguji model accuracy dari train yang dilakukan
#train_test_split akan mengambil secara random
#X_train = data untuk train ; Y_train = hasil untuk train ( anomaly ; normal )
#X_test = data untuk test ; Y_test = hasil untuk test ( anomaly ; normal )
X_train,X_test,Y_train,Y_test = train_test_split(train_x,train_y,train_size=0.70, random_state=2)

In [0]:
print(Y_train)

1312     anomaly
536      anomaly
12385    anomaly
24516    anomaly
19896     normal
          ...   
18898     normal
11798     normal
6637      normal
2575     anomaly
23720    anomaly
Name: class, Length: 17634, dtype: object


In [0]:
from sklearn.svm import SVC 
from sklearn.naive_bayes import BernoulliNB 
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import time
#proses learning dengan menggunakan macam macam methode

# Train KNeighborsClassifier Model
start_time1 = time.time()
KNN_Classifier = KNeighborsClassifier(n_jobs=-1)
KNN_Classifier.fit(X_train, Y_train); 
print("---KNN = %s seconds ---" % (time.time() - start_time1))

# Train LogisticRegression Model
start_time2 = time.time()
LGR_Classifier = LogisticRegression(n_jobs=-1, random_state=0)
LGR_Classifier.fit(X_train, Y_train);
print("---Logistic Regression = %s seconds ---" % (time.time() - start_time2))

# Train Gaussian Naive Baye Model
start_time3 = time.time()
BNB_Classifier = BernoulliNB()
BNB_Classifier.fit(X_train, Y_train)
print("---Naive Bayes = %s seconds ---" % (time.time() - start_time3))
            
# Train Decision Tree Model
start_time4 = time.time()
DTC_Classifier = tree.DecisionTreeClassifier(criterion='entropy', random_state=0)
DTC_Classifier.fit(X_train, Y_train)
print("---Decision Tree = %s seconds ---" % (time.time() - start_time4))
###############################################################

from sklearn import metrics

#menghitung dan membuat matric nilai
#menghitung accuracy dan melihat confusion matrix nya

print("###Pemisah#####")
#dari hasil perhitungan method terbaik adalah Decision Tree Classifier karena accuracy 100%

models = []
models.append(('Naive Baye Classifier', BNB_Classifier))
models.append(('Decision Tree Classifier', DTC_Classifier))
models.append(('KNeighborsClassifier', KNN_Classifier))
models.append(('LogisticRegression', LGR_Classifier))

for i, v in models:
    scores = cross_val_score(v, X_train, Y_train, cv=10)
    accuracy = metrics.accuracy_score(Y_test, v.predict(X_test))
    confusion_matrix = metrics.confusion_matrix(Y_test, v.predict(X_test))
    classification = metrics.classification_report(Y_test, v.predict(X_test))
    print()
    print('============================== {} Model Evaluation =============================='.format(i))
    print()
    print ("Cross Validation Mean Score:" "\n", scores.mean())
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()

---KNN = 0.5757615566253662 seconds ---
---Logistic Regression = 1.700916051864624 seconds ---
---Naive Bayes = 0.08258438110351562 seconds ---
---Decision Tree = 0.09077978134155273 seconds ---
###Pemisah#####

============================== Naive Baye Classifier Model Evaluation ==============================

Cross Validation Mean Score:
 0.9062592365363613

Model Accuracy:
 0.9053982535062186

Confusion matrix:
 [[2979  519]
 [ 196 3864]]

Classification report:
               precision    recall  f1-score   support

     anomaly       0.94      0.85      0.89      3498
      normal       0.88      0.95      0.92      4060

    accuracy                           0.91      7558
   macro avg       0.91      0.90      0.90      7558
weighted avg       0.91      0.91      0.90      7558



============================== Decision Tree Classifier Model Evaluation ==============================

Cross Validation Mean Score:
 0.9938186429799751

Model Accuracy:
 0.9943106641968775

Confusi